In [1]:
import numpy as np
import sys
sys.path.append( '../vqe-in-dft' )

import vqe_in_dft 

import scipy as sp
from pyscf import gto, dft, lib, mp, cc, scf, tools, ci, fci

In [2]:
geometry = [
['H', (0.7493682,0.0000000,0.4424329)],
['O', (0.0000000,0.0000000,-0.1653507)],
    
['H', (-0.7493682,0.0000000,0.4424329)]
]

# geometry = [
# ['H', (1.2473876659, -0.8998737590, 0.6150681570)],
# ['O', (1.2322305822, -0.2731895077, -0.1276123902)],

# ['C', (0.0849758188, 0.5590385475, 0.0510545434)],
# ['H', (0.1506137362, 1.1200249874, 0.9943015309)],
# ['H', (0.1316093068, 1.2841805400, -0.7645223601)],

# ['C', (-1.2129704155, -0.2295285634, -0.0097156258)],
# ['H', (-2.0801425360, 0.4329727646,0.0722817289)],
# ['H', (-1.2655910941, -0.9539857247, 0.8097953440)],
# ['H', (-1.2737541560, -0.7748626513, -0.9540587845)],
# ]

N_active_atoms = 2


low_level_scf_method='RKS'
high_level_scf_method='CCSD'
E_convergence_tol = 1e-6
basis = 'STO-3G' #'augccpvdz' # '6-31g' #'STO-3G'
unit= 'angstrom'
pyscf_print_level=1
memory=8000
charge=0
spin=0
run_fci= True#True
low_level_xc_functional = 'lda, vwn' # 'b3lyp'
high_level_xc_functional = 'b3lyp'

phys_notation = True

# Build the GLOBAL (full) molecule


and run supersystem calculation (cheap method)

https://pyscf.org/_modules/pyscf/gto/basis.html

In [3]:
full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       unit=unit,
                       spin=spin,
                      )
full_system_mol.build()
full_system_mol.atom

[['H', (0.7493682, 0.0, 0.4424329)],
 ['O', (0.0, 0.0, -0.1653507)],
 ['H', (-0.7493682, 0.0, 0.4424329)]]

In [4]:
## FCI comparison
HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.max_memory= memory
HF_scf.conv_tol = 1e-6
HF_scf.kernel()

my_fci = fci.FCI(HF_scf).run()
print('E(UHF-FCI) = %.12f' % my_fci.e_tot)

# myci = ci.CISD(HF_scf).run() # this is UCISD
# print('UCISD total energy = ', myci.e_tot)

E(UHF-FCI) = -75.015352478525


In [5]:
# Draw molecule

xyz_string = vqe_in_dft.Get_xyz_string(full_system_mol)
vqe_in_dft.Draw_molecule(xyz_string, width=400, height=400, jupyter_notebook=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
# Run global calculation

full_system_scf = scf.RKS(full_system_mol)
full_system_scf.verbose= pyscf_print_level
full_system_scf.max_memory= memory
full_system_scf.conv_tol = 1e-6
full_system_scf.xc = low_level_xc_functional
full_system_scf.kernel()

-74.73474041595246

# Localise orbitals

### Background

The overlap matrix is:

$$S_{\mu \nu} =  \int d\vec{r}_{1} \phi_{\mu}(1)^{*}\phi_{\nu}(1)$$

- $\phi_{\mu}$ are basis functions (defined in basis set)


The unknown molecular orbitals $\psi_{i}$ are expanded as a linear expansion of the $K$ known basis functions:

$$ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$$


$C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$. The columns of $C$ describe the molecular orbitals!

We can find the total number of electrons $N$ in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  2 \sum_{a}^{N/2} 1$$

- integral gives probablity of finding electron $a$ over all space (must be 1)
- summing over all electrons will give the total number of electrons

The charge density has the following definition:

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg)$$

- re-write using definition of $\psi_{i}=  \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu}$

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \Bigg( \bigg[ \sum_{\nu}^{K} C_{\nu a}^{*} \phi_{\nu}(\vec{r})^{*} \bigg] \bigg[ \sum_{\mu}^{K} C_{\mu a}\phi_{\mu}(\vec{r}) \bigg] \Bigg)$$

- move things around

$$\rho(\vec{r}) = \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$

- which is 

$$\rho(\vec{r}) = \sum_{\mu, \nu}^{K} P_{\mu \nu} \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$


- $P_{\mu \nu}$ is known as the density matrix and is:

$$P_{\mu \nu} = 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*}$$

Therefore we can also find the total number of electrons in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \int d\vec{r} \phi_{\mu}(\vec{r})  \phi_{\nu}(\vec{r})^{*}$$

- This is simply:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu}$$

- in an orthognal basis $S$ should be the identity matrix (as it is overlap of matrix of orthogonal orbs), SO WE GET:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu} = \sum_{\mu}^{K} PS_{\mu \mu} = \mathcal{Tr}(PS)$$

- One can interpret $ PS_{\mu \mu}$ in the above equation as the number of electrons associated with $ \phi_{\mu}$
- This is a **Mulliken population analysis**

# Orbital Localization!

A molecular orbital is usually delocalized, i.e. it has non-negligible amplitude over the whole system rather than only around some atom(s) or bond(s). However, one can choose a unitary rotation 

- When we perform a SCF calculation, one gets an optimized C matrix
    - $C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$
    - The columns of $C$ describe the molecular orbitals!
    - MO i: $ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$
    
    
- These molecular orbitals are usually **delocalized**
    - non-negligible amplitude over the whole system, rather than only around some atom(s) or bond(s)

- But we know in QM that a given basis choice is NOT unique


- We can therefore perform a unitary rotation on molecular orbitals

$$ \psi_{i} U_{rot} =  \Big( \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu} \Big) U_{rot} = \psi_{i}^{new}$$
    
    
The idea is to use a rotation such that the resulting orbitals $\psi_{i}^{new}$ are as spatially localized as possible. 


The Pipek-Mezey (PM) [localization](https://notendur.hi.is/hj/papers/paperPipekmezey8.pdf) **maximizes the population charges on the atoms**:

$$ f (U_{rot}) = \sum_{A}^{N_{atoms}} \Bigg( Z_{A} -  \sum_{\mu \text{ on atom } A} PS_{\mu \mu} \Bigg)$$

# Choose active and environment systems

## METHOD 1 

- Given optimized $C$ coefficient matrix
    - which has been rotated to localize orbitals
    - (used to build localized density matrix)


- **Look through basis functions $\phi_{\mu}$ of the ACTIVE atoms**
    
    
- check the mulliken charge // mulliken population of the orbital
    - if above a certain threshold associate it to active system
    - otherwise put in the environment

## METHOD 2 - SPADE

In [7]:
localization_method= 'ibo' # 'ibo', 'Boys', 'PipekMezey' 'SPADE'
THRESHOLD = 0.95

In [8]:
(C_active,
 C_envrio, 
 C_all_localized, 
 active_MO_inds,
 enviro_MO_inds) = vqe_in_dft.Localize_orbitals(localization_method, 
                                     full_system_scf, 
                                     N_active_atoms, 
                                     THRESHOLD=THRESHOLD, 
                                     sanity_check=True)


 Iterative localization: IB/P4/2x2, 5 iter; Final gradient 5.55e-19

overlap: [-0.02353434 -1.40488965 -0.69080156 -1.59840068  1.2031823 ]
threshold for active part: 0.95 

number of active MOs: 1
number of enviro MOs: 4 



In [9]:
# S_ovlp = full_system_scf.get_ovlp()
# AO_slice_matrix = full_system_scf.mol.aoslice_by_atom()
# ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])

# dm_test = 2* C_all_localized @ C_all_localized.conj().T
# # dm_test = 2* full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0] @ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0].conj().T

# DS = dm_test@S_ovlp

# # ao_active_inds
# diagonal_active = DS[ao_active_inds, ao_active_inds]

# diagonal_active


In [10]:
# S_ovlp = full_system_scf.get_ovlp()
# AO_slice_matrix = full_system_scf.mol.aoslice_by_atom()
# ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])

# S_half = sp.linalg.fractional_matrix_power(S_ovlp , 0.5)
# orthogonal_orbitals = (S_half@C_all_localized) # Get rows (the active AO) of orthogonal orbs 

# print(orthogonal_orbitals[:,2].dot(orthogonal_orbitals[:,3]))

# MO_AO_overlap = S_ovlp@orthogonal_orbitals
# MO_active_AO_overlap = np.einsum('ij->j', MO_AO_overlap[ao_active_inds]) 

# print('\noverlap:', MO_active_AO_overlap)
# print(f'threshold for active part: {THRESHOLD} \n')

# np.where(MO_active_AO_overlap>THRESHOLD)[0]

In [11]:
# Draw active orbitals

list_active_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
                             xyz_string,
                             C_all_localized, 
                             active_MO_inds,
                             width=400, 
                             height=400, 
                             jupyter_notebook=True)

list_enviro_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
                             xyz_string,
                             C_all_localized, 
                             enviro_MO_inds,
                             width=400, 
                             height=400, 
                             jupyter_notebook=True)

In [12]:
# list_active_orbitals[0]

In [13]:
# AO_slice_matrix = full_system_scf.mol.aoslice_by_atom()
# ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])
# ao_enviro_inds = np.arange(AO_slice_matrix[N_active_atoms,2], AO_slice_matrix[-1,-1])

# U, S, V = np.linalg.svd(C_all_localized)

# Projector_out_env = np.eye(C_all_localized.shape[0])
# Projector_out_env[ao_enviro_inds, ao_enviro_inds]= np.zeros((len(ao_enviro_inds),len(ao_enviro_inds)))
# Projector_out_env

In [14]:
dm_active, dm_enviro = vqe_in_dft.Get_active_and_envrio_dm(
                                                full_system_scf,
                                                C_active, 
                                                C_envrio, 
                                                C_all_localized, 
                                                sanity_check=True)

N_active_elec + N_environment_elec = N_total_elec is: True
y_active + y_enviro = y_total is: True


In [15]:
# use active density
E_act, J_act, K_act, e_xc_act, v_xc_act = vqe_in_dft.Get_energy_and_matrices_from_dm( full_system_scf, 
                                                                             dm_active, # <- ACTIVE
                                                                             check_E_with_pyscf=True)

In [16]:
# use enviro density
E_env, J_env, K_env, e_xc_env, v_xc_env = vqe_in_dft.Get_energy_and_matrices_from_dm(
                                                                            full_system_scf,
                                                                             dm_enviro, # <- ENVIRO
                                                                             check_E_with_pyscf=True)

In [17]:
# cross terms!
two_e_cross = vqe_in_dft.Get_cross_terms(full_system_scf, 
                                         dm_active, 
                                         dm_enviro, 
                                         J_env, 
                                         J_act, 
                                         e_xc_act,
                                         e_xc_env)

In [18]:
print(f'E_active: {E_act}')
print(f'E_enviro: {E_env}')
print(f'E_cross: {two_e_cross}')

E_active: -14.092327349306387
E_enviro: -81.0866796528154
E_cross: 13.431170954624172


# 4. Define V_embed

$$h^{A_{\text{act}} \text{ in } B_{\text{env}}} = h^{\text{core}} + v^{\text{embed}}$$

$$v^{\text{embed}} = g[\gamma^{A_{\text{act}}} + \gamma^{B_{\text{env}}} ] - g[\gamma^{A_{\text{act}}}] + P_{\text{projector}}$$

In [177]:
projector_method = 'huzinaga'
# projector_method = 'mu_shfit'

V_embed  = vqe_in_dft.Get_embedded_potential_operator(projector_method, 
                                full_system_scf, 
                                dm_active, 
                                dm_enviro, 
                                check_Hcore_is_correct=True, 
                                mu_shift_val=1e6,
                                check_Vemb=True)

H core is standard H_core


# 5. Run RKS DFT of full system with $V_{emb}$ to get $\gamma_{emb}^{\text{active}}$ 

In [179]:
full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= memory
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional

h_core = EMBEDDED_full_system_scf.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf.get_hcore = lambda *args: V_embed + h_core

E_emb = EMBEDDED_full_system_scf.kernel()

print(f'embedded Energy: {E_emb}')

embedded Energy: 6.353665401805102


In [158]:
EMBEDDED_full_system_scf.conv_check

True

In [180]:
EMBEDDED_occ_orbs = EMBEDDED_full_system_scf.mo_coeff[:,EMBEDDED_full_system_scf.mo_occ>0]

# optimized embedded denisty matrix
density_emb = 2 * EMBEDDED_occ_orbs @ EMBEDDED_occ_orbs.conj().T

## check number of electrons makes sense:
electron_check = np.isclose(np.trace(density_emb@full_system_scf.get_ovlp()), 2*len(active_MO_inds))

print(f'number of e- in gamma_embedded is correct: {electron_check}')

number of e- in gamma_embedded is correct: True


In [181]:
# calculate embedding correction term

dm_correction = np.einsum('ij, ij', V_embed, density_emb-dm_active)
WF_correction = np.einsum('ij, ij', V_embed, dm_active)

print(f'RKS correction: {dm_correction}')
print(f'WF correction: {WF_correction}')

RKS correction: -1.8800667021299844
WF correction: 13.57833346352092


In [182]:
## PsiEmbed Way

# J_emb, K_emb =EMBEDDED_full_system_scf.get_jk(dm=density_emb) 

# # note this uses the STANDARD H_core
# matrix_dot = lambda A, B: np.einsum('ij,ij', A, B)
# e_act_emb = matrix_dot(density_emb, h_core + 0.5 * J_emb - 0.25 * K_emb)
# e_act_emb

## MY way
e_act_emb = full_system_scf.energy_elec(dm=density_emb,
                                        vhf= full_system_scf.get_veff(dm=density_emb),
                                       h1e = h_core)[0]
e_act_emb

-14.472885087864878

In [143]:
e_mf_emb = e_act_emb + E_env + two_e_cross + full_system_scf.energy_nuc() + dm_correction
e_mf_emb # <-- energy from embedded DFT calc

-74.88017675990315

In [144]:
print(f'global DFT calculation == seperated calculation: {np.isclose(e_mf_emb, full_system_scf.e_tot)}')
# expected as same functional used!

global DFT calculation == seperated calculation: False


In [145]:
full_system_scf.e_tot

-74.73474041595246

# 6. High level DFT calc!

In [146]:
full_system_mol_HIGH_LEVEL_DFT = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )

full_system_mol_HIGH_LEVEL_DFT.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!
full_system_mol_HIGH_LEVEL_DFT.build()

full_system_scf_HIGH_LEVEL = scf.RKS(full_system_mol_HIGH_LEVEL_DFT)
full_system_scf_HIGH_LEVEL.verbose=1
full_system_scf_HIGH_LEVEL.max_memory= memory
full_system_scf_HIGH_LEVEL.conv_tol = 1e-6
full_system_scf_HIGH_LEVEL.xc = high_level_xc_functional # <-- BETTER functional!


# full_system_scf_HIGH_LEVEL.kernel() < --- do NOT RUN THIS

In [147]:
# run energy calc using high level functional (note density matrix is embedded A)

e_act_emb_HIGH_LVL = full_system_scf_HIGH_LEVEL.energy_elec(dm=density_emb)
e_act_emb_HIGH_LVL 

(-14.515700152112746, 0.830601962508047)

In [148]:
E_high_lvl_DFT = e_act_emb_HIGH_LVL[0] + E_env + two_e_cross + full_system_scf.energy_nuc() + dm_correction
E_high_lvl_DFT # <-- energy from embedded DFT calc

-74.92299182415101

In [183]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

High level DFT in DFT error: 0.09236065437380603
LOW level DFT in DFT error: 0.13517571862166733


# 7. High level WF calc (classical run) !

In [184]:
full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED) # <---- Hartree Fock Calc!
EMBEDDED_full_system_scf_HF.verbose=1
EMBEDDED_full_system_scf_HF.max_memory= memory
EMBEDDED_full_system_scf_HF.conv_tol = 1e-6

h_core = EMBEDDED_full_system_scf_HF.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: V_embed + h_core

# EMBEDDED_full_system_scf_HF.kernel() # <------ do NOT RUN!

In [185]:
# overwrite orbs with RKS embedded orbs!
EMBEDDED_full_system_scf_HF.mo_coeff = EMBEDDED_full_system_scf.mo_coeff 
EMBEDDED_full_system_scf_HF.mo_occ = EMBEDDED_full_system_scf.mo_occ 
EMBEDDED_full_system_scf_HF.mo_energy = EMBEDDED_full_system_scf.mo_energy

print(EMBEDDED_full_system_scf_HF.energy_elec(dm=density_emb)[0])
print(EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb))
print()
print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
print(EMBEDDED_full_system_scf_HF.energy_tot())

-2.715901733615167
6.41238199466777

-2.715901733615167
6.41238199466777


In [187]:
# EMBEDDED_full_system_scf_HF.kernel()
# FCI_embedded = fci.FCI(EMBEDDED_full_system_scf_HF)
# E_FCI,FCI_vec = FCI_embedded.kernel()
# # from pyscf.ci.cisd import  to_fcivec
# # cisdWFN   = to_fcivec(C_FCI, len(active_MO_inds), 2*len(active_MO_inds))

# RDM1 = FCI_embedded.make_rdm1(FCI_vec,
#                        norb=EMBEDDED_full_system_scf_HF.mo_coeff.shape[1],
#                        nelec=2*len(active_MO_inds))

# V,S, U = np.linalg.svd(RDM1)
# print(np.isclose(2*len(active_MO_inds), np.sum(S)))
# print(S)

# print('')
# print('ignore: ', S[])

In [56]:
## CCSD calculation

embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

embedded_cc_obj.frozen = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - len(enviro_MO_inds),
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]
# embedded_cc_obj.frozen  = enviro_MO_inds.tolist()

e_cc, t1, t2 = embedded_cc_obj.kernel()

CC_flag_check = np.isclose(EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb),
                          embedded_cc_obj.e_hf)

print(f'\nCC hartree fock energy matches HF embedded calc: {CC_flag_check}!')
embedded_cc_obj.e_hf

E(CCSD) = 6.375653448171412  E_corr = -0.03672854649635866

CC hartree fock energy matches HF embedded calc: True!


6.41238199466777

In [58]:
print('orbital energies:')
print(EMBEDDED_full_system_scf_HF.mo_energy)

orbital energies:
[-0.60880946 -0.5386189  -0.09700736 -0.04716088  0.14159934  0.61023583
 17.09380523]


In [61]:
print(embedded_cc_obj.frozen)

[3, 4, 5, 6]


In [62]:
WF_correction  = np.einsum('ij, ij', V_embed, dm_active) # note different definition

E_WF = embedded_cc_obj.e_hf +e_cc  + E_env + two_e_cross - WF_correction
E_WF 

-74.85818871354076

In [63]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))

High level DFT in DFT error: 0.09236065437380603
LOW level DFT in DFT error: 0.13517571862166733
WF in DFT error: 0.15716376498406248


In [64]:
drive_obj = vqe_in_dft.embeddeding_SCF_driver(geometry,
                 N_active_atoms,
                 projector_method,
                 cheap_global_SCF_method='RKS', 
                 cheap_global_DFT_xc= 'lda, vwn',
                 expensive_global_DFT_xc = 'b3lyp',
                 cheap_WF_method = 'RHF',
                 expensive_WF_method = 'CCSD',
                 E_convergence_tol = 1e-6,
                 basis = basis,
                 unit= 'angstrom',
                 pyscf_print_level=1,
                 memory=8000,
                 charge=0,
                 spin=0,
                 run_fci=False,
                 run_cisd=False,
                 mu_value=1e6,
                 physists_notation= phys_notation)

E_DFT_low, E_DFT_high, E_Classical, E_QC = drive_obj.run_experiment(localization_method, 
                                                                      orbtial_loc_threshold=THRESHOLD)

 Iterative localization: IB/P4/2x2, 5 iter; Final gradient 5.55e-19

overlap: [-0.02353434 -1.40488965 -0.69080156 -1.59840068  1.2031823 ]
threshold for active part: 0.95 

number of active MOs: 1
number of enviro MOs: 4 

N_active_elec + N_environment_elec = N_total_elec is: True
y_active + y_enviro = y_total is: True

number of e- in gamma_embedded is correct: True


RKS correction: -1.8800667021299844


global DFT calculation == seperated calculation: False
Cheap DFT energy Calculation: -74.88017675990315


Expensive DFT energy Calculation: -74.92299178523305

E(CCSD) = 6.375653448171412  E_corr = -0.03672854649635866

WF hartree fock energy matches HF embedded calc: True!


WF correction: 13.57833346352092
Expensive WF (classic) energy Calculation: -74.85818871354076

H core is NOT standard H_core
expect 2 electrons
quantum state has 3 electrons 

number of electrons correct: False
superposition states: [11 14 25 26 28 35 37 38 50 52]


In [70]:
# np.binary_repr(52).count('1')  

In [71]:
print('LOW level DFT in DFT error:', np.abs(E_DFT_low-my_fci.e_tot))
print('High level DFT in DFT error:', np.abs(E_DFT_high-my_fci.e_tot))
print('WF in DFT error:', np.abs(E_Classical-my_fci.e_tot))
print('VQE in DFT error:', np.abs(E_QC-my_fci.e_tot))

LOW level DFT in DFT error: 60.54246739065994
High level DFT in DFT error: 0.09236069329176644
WF in DFT error: 0.15716376498406248
VQE in DFT error: 0.04200443233688134


In [41]:
print(e_mf_emb)
print(E_high_lvl_DFT)
print(E_WF)

-74.88017675990315
-74.92299182415101
-74.85818871354076


In [42]:
fdasd

NameError: name 'fdasd' is not defined

# QC part

$$H_{fermionic} = h_{nuc} + \sum_{p, q} h_{pq} a_{p}^{\dagger} a_{q} + \frac{1}{2} \sum_{p, q ,r ,s} h_{pqrs} a_{p}^{\dagger} a_{q}^{\dagger} a_{r} a_{s}$$

where:

- $$h_{pq} = \int d{\vec{x}} \phi_{p}^{*}({\vec{x}}) \bigg( - \frac{\nabla^{2}_{{\vec{r}}}}{2} - \sum_{I} \frac{Z_{I}}{|{\vec{r}} - {\vec{R}_{I}} |} \bigg) \phi_{q}({\vec{x}})$$


- $$h_{pqrs} = \int d{\vec{x}}_{1} d{\vec{x}}_{2} \frac{\phi_{p}^{*}({\vec{x}}_{1}) \phi_{q}^{*}({\vec{x}}_{2}) \phi_{s}({\vec{x}}_{1}) \phi_{r}({\vec{x}}_{2})}{|{\vec{r}_{1}} - {\vec{r}}_{2}|}$$ 


- $$h_{nuc} = \frac{1}{2} \sum_{I \neq J} \frac{Z_{I} Z_{J}}{| {\vec{R}}_{I} - {\vec{R}}_{J}|}$$

note project means we don't want last set of MO orbs filled!

In [ ]:
EMBEDDED_full_system_scf_HF.mo_coeff[:]

In [ ]:
N_enviroment_MOs = len(enviro_MO_inds) 

one_body_integrals, two_body_integrals = vqe_in_dft.Get_embedded_one_and_two_body_integrals_MO_basis(EMBEDDED_full_system_scf_HF,
                                                                                            N_enviroment_MOs,
                                                                                            physists_notation=phys_notation)

In [ ]:
print(one_body_integrals.shape)
print(two_body_integrals.shape)

print(active_MO_inds)
print(enviro_MO_inds)

In [ ]:
one_body_terms, two_body_terms = vqe_in_dft.Get_SpinOrbs_from_Spatial(one_body_integrals,
                                                                   two_body_integrals,
                                                                   physists_notation=phys_notation,
                                                                   EQ_Tolerance=1e-8)

In [ ]:
Nuclear_energy =  full_system_scf.energy_nuc()

H_fermionic = vqe_in_dft.Get_fermionic_H(one_body_terms, 
                                     two_body_terms, 
                                     Nuclear_energy,
                                     core_constant=0, 
                                     physists_notation=phys_notation)

In [ ]:
len(list(H_fermionic))

In [ ]:
from openfermion.linalg import get_sparse_operator
H_sparse = get_sparse_operator(H_fermionic)

In [ ]:
eigvals_EMBED, eigvecs_EMBED = sp.sparse.linalg.eigsh(H_sparse, which='SA', k=1)
eigvals_EMBED

In [ ]:
WF_correction  = np.einsum('ij, ij', V_embed, dm_active) # note different definition

E_VQE = eigvals_EMBED[0]  + E_env + two_e_cross - WF_correction
E_VQE 

In [ ]:
N_electrons_expected = 2*len(active_MO_inds)
N_electrons_Q_state = np.binary_repr(np.where(np.abs(eigvecs_EMBED)>1e-2)[0][0]).count('1') 

print(f'expect {N_electrons_expected} electrons')
print(f'quantum state has {N_electrons_Q_state} electrons \n')

print(f'number of electrons correct: {N_electrons_expected == N_electrons_Q_state}')

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))


print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))
print('VQE error:', np.abs(E_VQE-my_fci.e_tot))

In [ ]:
where=np.where(np.around(eigvecs_EMBED, 4)>0)[0]
print(where)

np.binary_repr(where[0]).count('1') 